# Web Scraping das Top 250 Séries do IMDb

Este notebook realiza um scraping passo a passo em **Selenium** para:
1. Extrair as 250 séries mais bem avaliadas no IMDb.
2. Visitar cada página de série e coletar popularidade, elenco principal e número total de episódios.
3. Salvar os resultados em **JSON**.


## 1. Instalação de dependências

Execute no terminal:

```bash
pip install selenium webdriver-manager
```

## 2. Importar bibliotecas e inicializar o navegador

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import json
import time

# Inicializar o Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


## 3. Capturar as 250 séries mais bem avaliadas

In [ ]:
url = "https://www.imdb.com/chart/toptv/"
driver.get(url)
time.sleep(2)

series_list = []
# Seletor dos itens de série
rows = driver.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")
for row in rows:
    try:
        title_elem = row.find_element(By.CSS_SELECTOR, "h3 a")
        title_raw = title_elem.text.strip()
        # Remove dígitos do título (ex.: ano embutido)
        title = ''.join([c for c in title_raw if not c.isdigit()]).strip(". ")

        link = title_elem.get_attribute("href")
        year = int(row.find_element(By.CSS_SELECTOR, 
                    ".sc-b189961a-8.kLaxqf.cli-title-metadata-item").text.strip("()"))
        rating = float(row.find_element(By.CSS_SELECTOR, 
                    "span.sc-bde20123-1.cMEQkK").text.strip())

        series_list.append({
            "Título": title,
            "Ano": year,
            "Nota IMDB": rating,
            "Link": link
        })
    except Exception as e:
        print(f"Erro ao processar linha: {e}")

print(f"Capturadas {len(series_list)} séries.")

## 4. Visitar cada série e coletar detalhes adicionais

In [ ]:
for serie in series_list:
    driver.get(serie["Link"])
    time.sleep(1.5)
    try:
        # Número total de episódios
        episodes = driver.find_element(By.CSS_SELECTOR, 
                   'li[data-testid="episodes-header"] span').text
        serie["Total de Episódios"] = int(
            episodes.replace(" episódios", "").replace(",", ""))
        
        # Popularidade
        popularity = driver.find_element(By.CSS_SELECTOR, 
                   'div[data-testid="hero-rating-bar__popularity__score"]').text
        serie["Popularidade"] = int(
            popularity.replace("#", "").replace(",", ""))

        # Elenco principal (até 10 integrantes)
        elenco_elems = driver.find_elements(By.CSS_SELECTOR, 
                       'section[data-testid="title-cast"] li.ipc-metadata-list-summary-item')
        elenco = []
        for ator in elenco_elems[:10]:
            try:
                nome = ator.find_element(By.CSS_SELECTOR, 
                           'a.ipc-metadata-list-summary-item__t').text.strip()
                pers = ator.find_element(By.CSS_SELECTOR, 
                       'div.ipc-metadata-list-summary-item__supplemental').text.strip()
                elenco.append({"Ator/Atriz": nome, "Personagem": pers})
            except:
                continue
        serie["Elenco Principal"] = elenco

    except Exception as e:
        print(f"Erro ao processar {serie['Título']}: {e}")

## 5. Salvar resultados e encerrar

In [ ]:
with open("series_imdb_top250.json", "w", encoding="utf-8") as f:
    json.dump(series_list, f, indent=4, ensure_ascii=False)

driver.quit()
print("Arquivo salvo: series_imdb_top250.json")